In [181]:
import numpy as np
import math
# Get k-mers from a sequence


def get_kmers(sequence, k):
    return [sequence[x:x+k] for x in range(len(sequence) - k + 1)]

# Get the frequency of each k-mer in a sequence


def freq_kmers(seq, k):
    kmers = get_kmers(seq, k)
    freq = {}
    for kmer in kmers:
        if kmer not in freq:
            freq[kmer] = 1
        else:
            freq[kmer] += 1
    return freq


In [182]:
# Implement Spectrum Kernel
def spectrum_kernel(seq1, seq2, k):
    possiblekmers = kmers_generator(k)
    freq1 = freq_kmers(seq1, k)
    freq2 = freq_kmers(seq2, k)
    for key in possiblekmers:
        if key not in freq1:
            freq1[key] = 0
        if key not in freq2:
            freq2[key] = 0
            
    freq1_lst = [freq1[key] for key in possiblekmers]
    freq2_lst = [freq2[key] for key in possiblekmers]
    similarity = np.dot([freq1[key] for key in possiblekmers], [freq2[key] for key in possiblekmers])
    normalization = math.sqrt(np.dot([freq1[key] for key in freq1],[freq1[key] for key in freq1])) *  math.sqrt(np.dot([freq2[key] for key in freq1],[freq2[key] for key in freq1]))
    # freq1 = {k: v for k, v in sorted(freq1.items(), key=lambda item: item[1], reverse=True)}
    # freq2 = {k: v for k, v in sorted(freq2.items(), key=lambda item: item[1], reverse=True)}
    #similarity = similarity/normalization
    return similarity, freq1_lst, freq2_lst


In [183]:
# change fasta file into a list of sequences
# fasta reader
def read_fasta(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    return lines

# fasta parser


def parse_fasta(lines):
    seq_list = []
    seq_name = [line for line in lines if line.startswith('>')]
    seq = ''
    for line, index in zip(lines, range(len(lines))):
        if index == len(lines) - 1:
            seq += line.strip()
            seq_list.append(seq)
        if line.startswith('>'):
            seq_list.append(seq)
            seq = ''
            continue
        else:
            seq += line.strip()
    for i in seq_list:
        if i == '':
            seq_list.remove(i)
    return seq_list, seq_name


In [184]:
seq, seq_name = parse_fasta(read_fasta('./kmeans/kmeans.fasta'))
# print(spectrum_kernel(seq1, seq2, 3))
print(seq_name)


FileNotFoundError: [Errno 2] No such file or directory: './kmeans/kmeans.fasta'

In [ ]:
seq1 = "AATCCG"
seq2 = "AATGCC"
print(spectrum_kernel(seq1, seq2, 2))


3


In [ ]:
# KNN
def dist(seq1, seq2, k):
    return np.sqrt(spectrum_kernel(seq1, seq1, k)-2*spectrum_kernel(seq1, seq2, k)+spectrum_kernel(seq2, seq2, k))


def KNN(train, test, knn_num, kmer_size):
    seq_train = []
    seq_name_train = []
    for path in train:
        seq, seq_name = parse_fasta(read_fasta(path))
        seq_train += seq
        seq_name_train += seq_name
    seq_test, seq_name_test = parse_fasta(read_fasta(test))
    train_class_list = ["exons" if "exon" in seq_name_train[i]
                        else "introns" for i in range(len(seq_name_train))]
    test_true_class_list = ["exons" if "exon" in seq_name_test[i]
                            else "introns" for i in range(len(seq_name_test))]
    test_pred_class_list = []
    for i in range(len(seq_test)):
        dist_list = []
        for j in range(len(seq_train)):
            dist_list.append(dist(seq_test[i], seq_train[j], kmer_size))
        dist_list = np.array(dist_list)
        indexes = np.argsort(dist_list)
        exons_num = 0
        introns_num = 0
        for index in indexes[:knn_num]:
            if train_class_list[index] == "exons":
                exons_num += 1
            else:
                introns_num += 1
        if exons_num > introns_num:
            test_pred_class_list.append("exons")
        else:
            test_pred_class_list.append("introns")
    correct = 0
    for i, j in zip(test_true_class_list, test_pred_class_list):
        if i == j:
            correct += 1

    accuracy = correct/len(test_true_class_list)

    return accuracy


In [ ]:
KNN(train=["./KNN/train-exons100.fasta", "./KNN/train-introns100.fasta"],
    test="./KNN/test.fasta", knn_num=5, kmer_size=2)


0.97375

In [ ]:
# change fasta file into a list of sequences
# fasta reader
def read_fasta(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    return lines

# fasta parser


def parse_fasta(lines):
    seq_list = []
    seq = ''
    seq_name = []
    for line, index in zip(lines, range(len(lines))):
        if index == len(lines) - 1:
            seq += line.strip()
            seq_list.append(seq)
        if line.startswith('>'):
            seq_list.append(seq)
            seq = ''
            name = line.strip()
            seq_name.append(name.split(" /")[0][1:])
            continue
        else:
            seq += line.strip()
    for i in seq_list:
        if i == '':
            seq_list.remove(i)
    return seq_list, seq_name


In [ ]:
def diff(key1, key2):
    list1 = list(key1)
    list2 = list(key2)
    dif = 0
    for i in range(len(list1)):
        if list1[i] != list2[i]:
            dif += 1
    return dif


# KMEANS

In [ ]:
def generate_mismatch_kmers(kmer):
    bases = ['A', 'C', 'G', 'T']
    mismatches = []
    for i in range(len(kmer)):
        for b in bases:
            if b != kmer[i]:
                new_kmer = kmer[:i] + b + kmer[i+1:]
                mismatches.append(new_kmer)
    return mismatches
def kmers_generator(k):
    bases = ['A', 'C', 'G', 'T']
    kmers = []
    for i in range(k):
        if i == 0:
            kmers = bases
        else:
            kmers = [kmer + b for kmer in kmers for b in bases]
    return kmers


def count_kmer(seq, kmer):
    count = 0
    k = len(kmer)
    for i in range(len(seq) - k + 1):
        if seq[i:i+k] == kmer:
            count += 1
    return count

def mismatch_freq_kmers(seq, k):
    kmers = kmers_generator(k)
    freq = {}
    # count all kmers
    for kmer in kmers:
        freq[kmer] = count_kmer(seq, kmer)
    # count all mismatch kmers
    for i in range(len(seq) - k + 1):
        mismatch_kmers = generate_mismatch_kmers(seq[i:i+k])
        for mismatch_kmer in mismatch_kmers:
                freq[mismatch_kmer] += 1
    return freq

def mismatch_spectrum_kernel(seq1, seq2, k):
    freq1 = mismatch_freq_kmers(seq1, k)
    freq2 = mismatch_freq_kmers(seq2, k)
    for key in freq1:
        if key not in freq2:
            freq2[key] = 0
    for key in freq2:
        if key not in freq1:
            freq1[key] = 0
    freq1_lst = [freq1[key] for key in freq1]
    freq2_lst = [freq2[key] for key in freq1]
    similarity = np.dot(freq1_lst, freq2_lst)
    normalization = math.sqrt(np.dot(freq1_lst, freq1_lst)) * math.sqrt(np.dot(freq2_lst, freq2_lst))
    

    return similarity, freq1_lst, freq2_lst


In [ ]:
len(kmers_generator(6))

4096

In [ ]:
def kmeans(seqpath, ncluster, kmer_size, maxniter, distfunc):
    seq_list, seq_names = parse_fasta(read_fasta(seqpath))
    centroids = np.random.randint(0, len(seq_list), ncluster)
    # initialize class_list
    class_list = {}
    seq_kernel_list = []
    for seq in seq_list:
        similarity_list = []
        for centroid in centroids:
            similarity, seq_spectrum, centroid_spectrum = distfunc(
                seq, seq_list[centroid], kmer_size)
            similarity = np.dot(centroid_spectrum,centroid_spectrum) + np.dot(seq_spectrum,seq_spectrum) - 2 * similarity
            seq_kernel_list.append(seq_spectrum)
            similarity_list.append(similarity)

        class_list[seq] = np.argmin(np.array(similarity_list))
    # update centroids 1st time
    centroids = []
    for i in range(ncluster):
        tmp_list = []
        for seq, seq_kernel in zip(seq_list, seq_kernel_list):
            if class_list[seq] == i:
                tmp_list.append(seq_kernel)
        centroids.append(np.mean(tmp_list, axis=0))
    centroids_new = centroids.copy()
    old_class_list = class_list.copy()
    for i in range(maxniter):
        # update class_list
        for seq, seq_kernel in zip(seq_list, seq_kernel_list):
            similarity_list = []
            for centroid in centroids_new:
                similarity = np.dot(seq_kernel, centroid)
                similarity = np.dot(centroid_spectrum,centroid_spectrum) + np.dot(seq_spectrum,seq_spectrum) - 2 * similarity
                similarity_list.append(similarity)
            class_list[seq] = np.argmin(np.array(similarity_list))
        if class_list == old_class_list:
            break
        old_class_list = class_list.copy()
        # update centroids
        centroids_new = []
        for i in range(ncluster):
            tmp_list = []
            for seq, seq_kernel in zip(seq_list, seq_kernel_list):
                if class_list[seq] == i:
                    tmp_list.append(seq_kernel)
            centroids_new.append(np.mean(tmp_list, axis=0))

    return centroids_new, class_list


In [ ]:
seqpath = "kmeans.fasta"
#centroids, class_list = kmeans(seqspath, 3, 6, 100, mismatch_spectrum_kernel)
# centroids

In [186]:
seq_list, seq_names = parse_fasta(read_fasta(seqspath))
ncluster = 2
kmer_size = 6
maxniter = 100
centroids, class_list = kmeans(seqpath, ncluster, kmer_size, maxniter, mismatch_spectrum_kernel)


In [ ]:
def printtest(seq_list,seq_names,class_list,kmer_size,ncluster,title):
    print('\n', title, ' (KMER =', kmer_size, ", CLUSTERS =", ncluster, "):")
    for i in range(ncluster):
        exon = 0
        intron = 0
        intergenic = 0
        print('\n', 'CLUSTER ',i + 1,":")
        for seq, annotation in zip(seq_list,seq_names):
            if class_list[seq] == i:
                if "exon" in annotation:
                    exon += 1
                if "intron" in annotation:
                    intron += 1
                if "intergenic" in annotation:
                    intergenic += 1
        sum = intergenic + exon + intron
        print('\n','    intergenic =', round(intergenic/sum,2),' (', intergenic, ')')
        print('\n','    intron =', round(intron/sum,2),' (', intron, ')')
        print('\n','    exon =', round(exon/sum,2),' (', exon, ')')

In [189]:
report_V2(seq_list,seq_names,class_list,kmer_size,ncluster,"MISMATCH KERNEL")

In [194]:
seq_list, seq_names = parse_fasta(read_fasta(seqspath))
ncluster = 5
kmer_size = 6
maxniter = 2
centroids, class_list = kmeans(seqpath, ncluster, kmer_size, maxniter, spectrum_kernel)


In [196]:
report_V2(seq_list,seq_names,class_list,kmer_size,ncluster,"MISMATCH KERNEL")

In [ ]:
printtest(seq_list,seq_names,class_list,kmer_size,ncluster,"MISMATCH KERNEL")

In [188]:
def report_V2(seq_list,seq_names,class_list,kmer_size,ncluster,title):
    file = open("KmeansResults.txt", "a")
    file.write( '\n' +title + ' (KMER =' + str(kmer_size) + ", CLUSTERS =" + str(ncluster) + "):\n")
    for i in range(ncluster):
        exon = 0
        intron = 0
        intergenic = 0
        file.write('CLUSTER ' + str(i + 1) + ":\n")
        for seq, annotation in zip(seq_list,seq_names):
            if class_list[seq] == i:
                if "exon" in annotation:
                    exon += 1
                if "intron" in annotation:
                    intron += 1
                if "intergenic" in annotation:
                    intergenic += 1
        sum = intergenic + exon + intron
        file.write('    intergenic =' + str(round(intergenic/sum,2)) + ' (' + str(intergenic) +  ')\n')
        file.write('    intron =' + str(round(intron/sum,2)) + ' (' + str(intron) +  ')\n')
        file.write('    exon =' + str(round(exon/sum,2)) + ' (' + str(exon) +  ')\n')

 

In [137]:
seq_list, seq_names = parse_fasta(read_fasta(seqpath))
centroids = np.random.randint(0, len(seq_list), ncluster)
    # initialize class_list
class_list = {}
seq_kernel_list = []
for seq in seq_list:
    similarity_list = []
    for centroid in centroids:
        similarity, seq_spectrum, centroid_spectrum = spectrum_kernel(
                seq, seq_list[centroid], kmer_size)
        similarity = np.dot(centroid_spectrum,centroid_spectrum) + np.dot(seq_spectrum,seq_spectrum) - 2 * similarity
        seq_kernel_list.append(seq_spectrum)
        similarity_list.append(similarity)

    class_list[seq] = np.argmin(np.array(similarity_list))
    # update centroids 1st time
centroids = []
for i in range(ncluster):
    tmp_list = []
    for seq, seq_kernel in zip(seq_list, seq_kernel_list):
        if class_list[seq] == i:
            tmp_list.append(seq_kernel)
    
    centroids_new.append(np.mean(tmp_list, axis=0))

    

In [143]:
len(kmers_generator(4))

256

In [129]:
for i in range(maxniter):
        # update class_list
    for seq, seq_kernel in zip(seq_list, seq_kernel_list):
        similarity_list = []
        for centroid in centroids_new:
            similarity = np.dot(seq_kernel, centroid)
            similarity = np.dot(centroid_spectrum,centroid_spectrum) + np.dot(seq_spectrum,seq_spectrum) - 2 * similarity
            similarity_list.append(similarity)
        class_list[seq] = np.argmin(np.array(similarity_list))
    if class_list == old_class_list:
        break
    old_class_list = class_list.copy()
        # update centroids
    centroids_new = []
    for i in range(ncluster):
        tmp_list = []
        for seq, seq_kernel in zip(seq_list, seq_kernel_list):
            if class_list[seq] == i:
                tmp_list.append(seq_kernel)
        centroids_new.append(np.mean(tmp_list, axis=0))

ValueError: shapes (256,) and (61878,) not aligned: 256 (dim 0) != 61878 (dim 0)

In [157]:
freq_kmers(seq_list[1], 2)

{'GA': 11,
 'AG': 17,
 'GT': 11,
 'TA': 41,
 'AA': 55,
 'AT': 43,
 'TT': 34,
 'AC': 15,
 'CT': 11,
 'TC': 15,
 'TG': 9,
 'CA': 23,
 'CC': 9,
 'GC': 5}

In [158]:
spectrum_kernel(seq_list[1], seq_list[2], 2)

(8690,
 [55, 15, 17, 43, 23, 9, 0, 11, 11, 5, 0, 11, 41, 15, 9, 34],
 [59, 12, 11, 34, 24, 9, 2, 10, 13, 7, 9, 14, 20, 16, 21, 38])